## Examen Parcial CC0C2

**Nombre y Apellidos: RICARDO LEONARDO OLIVARES VENTURA**

**Código: 20192002A**

### Reglas para el Examen Parcial

- Queda terminantemente prohibido el uso de herramientas como ChatGPT, WhatsApp, o cualquier herramienta similar durante la realización de esta prueba. El uso de estas herramientas, por cualquier motivo, resultará en la anulación inmediata de la evaluación. Puedes utilizar los cuadernos y datos alojados en github.

- Las respuestas deben presentarse con una explicación detallada, utilizando términos técnicos apropiados. La mera descripción sin el uso de terminología técnica, especialmente términos discutidos en clase, se considerará insuficiente y podrá resultar en que la respuesta sea marcada como incorrecta.

- Cada estudiante debe presentar su propio trabajo. Los códigos iguales o muy parecidos entre sí serán considerados como una violación a la integridad académica, implicando una copia, y serán sancionados de acuerdo con las políticas de la universidad.

- Todos los estudiantes deben subir sus repositorios de código a la plataforma del curso, según las instrucciones proporcionadas. La fecha y hora de la última actualización del repositorio serán consideradas como la hora de entrega.

- La claridad, orden, y presentación general de las evaluaciones serán tomadas en cuenta en la calificación final. Se espera un nivel de profesionalismo en la documentación y presentación del código y las respuestas escritas.


#### Instrucciones de entrega para la prueba calificada

- Presenta la dirección de tu repositorio personal donde se encuentre este cuaderno con tus respuestas desarrolladas.
- Todo cambio fuera de la hora y fecha del examen realizado dentro del repositorio no se tomará en cuenta y se procederá a anular la evaluación.

### Problema 1

El subsampling en el contexto de los modelos de Word2Vec es una técnica utilizada para reducir el número de veces que se entrenan palabras muy frecuentes. Se basa en la idea de que las palabras extremadamente comunes (como preposiciones y conjunciones) proporcionan menos información de contexto valiosa en comparación con las palabras menos frecuentes. En la práctica, cada palabra en el conjunto de entrenamiento tiene una probabilidad calculada de ser "saltada" durante el entrenamiento, dependiendo de su frecuencia. Esto ayuda a acelerar el entrenamiento y a mejorar la calidad de las representaciones de palabras menos frecuentes, que podrían verse oscurecidas por palabras de alta frecuencia.

El negative sampling es una técnica de optimización para reducir la complejidad computacional de actualizar los pesos en la red neuronal en modelos como Word2Vec. En lugar de actualizar los pesos de todas las palabras del vocabulario para cada ejemplo de entrenamiento (lo cual es muy costoso computacionalmente), el negative sampling actualiza solo un pequeño número de "palabras negativas" (ejemplos negativos seleccionados aleatoriamente) junto con la palabra objetivo (ejemplo positivo). Esto no solo acelera significativamente el entrenamiento sino que también mejora la calidad de las representaciones vectoriales al enfocarse en distinguir la palabra objetivo de un pequeño subconjunto de palabras negativas.

La correlación de Spearman es una medida estadística que evalúa la fuerza y la dirección de la asociación entre dos variables clasificadas. A diferencia de la correlación de Pearson, que requiere que las variables sean de escala intervalo o de razón y aproximadamente normales, la correlación de Spearman no hace suposiciones sobre la distribución de los datos y se basa en rangos. Es especialmente útil en el contexto de Word2Vec cuando se evalúa cómo las similitudes coseno calculadas entre vectores de palabras se comparan con juicios humanos de similitud (usualmente dados en estudios donde las personas califican qué tan similares son las palabras). Al correlacionar estos dos conjuntos de rankings (el calculado y el humano), se puede obtener una medida de cuán bien el modelo captura relaciones semánticas que coinciden con las percepciones humanas.

#### Ejercicios:

1. Implementa los modelos CBOW y Skip-gram en Python sin utilizar bibliotecas de alto nivel como Gensim (2 puntos).
    - Escribe el código para inicializar los pesos de la red, realizar el entrenamiento mediante descenso de gradiente y calcular la función de pérdida.
    - Añade mecanismos de subsampling y negative sampling para mejorar la eficiencia del entrenamiento.
2. Analiza cómo diferentes hiperparámetros afectan la calidad de los embeddings vectoriales (2 puntos).
    - Entrena modelos Word2Vec con diferentes tamaños de ventana, dimensiones de vector y tasas de aprendizaje. Utiliza un conjunto de datos estándar como el corpus de texto de Wikipedia.
    - Evalúa los modelos usando tareas de analogía de palabras y calcula la correlación de Spearman entre las similitudes humanas y las  calculadas por el modelo.



In [ ]:
## Tus respuestas
import numpy as np

# 1 .Implemente los modelos CBOW y Skip-gram

# 1.a. Escribe el código para inicializar los pesos de lan red,
# realizar el entrenamiento mediante descenso de gradiente y
# calcular la función de pérdida

#CBOW:
# Este modelo se encarga de predecir una palabra objetivo basándose
# en las palabras del contexto circundantes
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class CBOW:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        # Inicializamo los pesos de red
        self.W1 = np.random.rand(vocab_size, embedding_dim)
        self.W2 = np.random.rand(embedding_dim, vocab_size)

    def train(self, context, target, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            h = np.mean(self.W1[context], axis=0)
            u = np.dot(h, self.W2)
            y_pred = softmax(u)

            # Error
            EI = np.array(y_pred)
            EI[target] -= 1

            # Backpropagacion
            dW2 = np.outer(h, EI)
            dW1 = np.dot(self.W2, EI).reshape(self.W1[context].shape)

            self.W1[context] -= learning_rate * dW1
            self.W2 -= learning_rate * dW2

            if epoch % 100 == 0:
              # Aquí también mostramos la pérdida que se efectúa en cada época
                print(f'Epoca {epoch}, Perdida: {np.sum(-np.log(y_pred[target]))}')

    def word_vector(self, word_idx):
        return self.W1[word_idx]


# Skip-Gram:
# Es lo opuesto al CBOW, ya que, este modelo intenta predecir las palabras
# contexto a partir de la palabra objetivo

class SkipGram:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        # Inicializamos los pesos de red
        self.W1 = np.random.rand(vocab_size, embedding_dim)
        self.W2 = np.random.rand(embedding_dim, vocab_size)

    def train(self, target, contexts, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            h = self.W1[target]
            u = np.dot(h, self.W2)
            y_pred = softmax(u)

            EI = np.array(y_pred)
            EI[contexts] -= 1 / len(contexts)

            dW2 = np.outer(h, EI)
            dW1 = np.dot(self.W2, EI).reshape(self.W1[target].shape)

            self.W1[target] -= learning_rate * dW1
            self.W2 -= learning_rate * dW2

            if epoch % 100 == 0:
              # Función de pérdida
                print(f'Epoca {epoch}, Perdida: {np.sum(-np.log(y_pred[contexts]))}')

    def word_vector(self, word_idx):
        return self.W1[word_idx]







ValueError: cannot reshape array of size 5 into shape (4,5)

In [ ]:
# 2. Analiza cómo diferentes hiperparámetros afectan la calidad de los embeddings vectoriales (2 puntos).
# Entrena modelos Word2Vec con diferentes tamaños de ventana, dimensiones de vector y tasas de aprendizaje. Utiliza
# un conjunto de datos estándar como el corpus de texto de Wikipedia.
# Evalúa los modelos usando tareas de analogía de palabras y calcula la correlación de Spearman
# entre las similitudes humanas y las calculadas por el modelo.

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import numpy as np
from scipy.stats import spearmanr

# Función para cargar y limpiar el corpus de texto
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        corpus = f.read()
    # Aquí puedes agregar la lógica para limpiar y preprocesar el texto si es necesario
    return corpus

# Función para tokenizar el corpus
def tokenize_corpus(corpus):
    # Aquí puedes usar tu método preferido para dividir el texto en oraciones o párrafos
    tokenized_corpus = [sentence.split() for sentence in corpus.split('\n')]
    return tokenized_corpus

# Función para entrenar modelos Word2Vec con diferentes hiperparámetros
def train_word2vec_model(tokenized_corpus, window_size, vector_size, learning_rate):
    model = Word2Vec(sentences=tokenized_corpus, vector_size=vector_size, window=window_size, sg=1, alpha=learning_rate, min_alpha=0.0001, seed=42)
    return model

# Función para evaluar modelos utilizando tareas de analogía de palabras
def evaluate_model(model, analogy_dataset):
    correct_predictions = 0
    total_predictions = 0
    model_predictions = []

    for analogy in analogy_dataset:
        try:
            predicted_words = model.wv.most_similar(positive=[analogy[1], analogy[2]], negative=[analogy[0]], topn=1)
            model_predictions.append(predicted_words[0][0])
            if predicted_words[0][0] == analogy[3]:
                correct_predictions += 1
            total_predictions += 1
        except KeyError:
            continue

    # Calcula la correlación de Spearman entre las similitudes humanas y las predicciones del modelo
    spearman_correlation, _ = spearmanr([1 if prediction == analogy[3] else 0 for prediction in model_predictions], range(len(model_predictions)))
    return correct_predictions / total_predictions, spearman_correlation

# Corpus de texto de Wikipedia (reemplaza esto con la ruta a tu propio archivo)
corpus_path = "wikipedia_corpus.txt"
# Datos de analogía de palabras (reemplaza esto con tu propio conjunto de datos)
analogy_dataset = [
    ("man", "king", "woman", "queen"),
    ("france", "paris", "germany", "berlin"),
    ("man", "doctor", "woman", "nurse")
]

# Cargar y tokenizar el corpus
corpus = load_corpus(corpus_path)
tokenized_corpus = tokenize_corpus(corpus)

# Hiperparámetros a probar
window_sizes = [5, 10, 15]
vector_sizes = [100, 200, 300]
learning_rates = [0.025, 0.05, 0.1]

# Bucle para probar diferentes combinaciones de hiperparámetros
for window_size in window_sizes:
    for vector_size in vector_sizes:
        for learning_rate in learning_rates:
            # Entrenar modelo
            model = train_word2vec_model(tokenized_corpus, window_size, vector_size, learning_rate)
            # Evaluar modelo
            accuracy, spearman_corr = evaluate_model(model, analogy_dataset)
            print(f"Window Size: {window_size}, Vector Size: {vector_size}, Learning Rate: {learning_rate}, Accuracy: {accuracy}, Spearman Correlation: {spearman_corr}")


ZeroDivisionError: division by zero

### Pregunta 2

La factorización de matrices GloVe y PPMI son dos métodos utilizados en el procesamiento del lenguaje natural (NLP) para capturar relaciones semánticas entre palabras a partir de grandes corpus de texto. Ambos métodos se utilizan para generar representaciones vectoriales de palabras, lo que permite que las relaciones semánticas y sintácticas entre palabras se reflejen en el espacio vectorial.

1 . GloVe (Global Vectors for Word Representation)
GloVe es un modelo de aprendizaje no supervisado para obtener representaciones vectoriales de palabras. Fue desarrollado por investigadores de Stanford y combina elementos de dos enfoques principales en NLP: factorización de matrices y modelos basados en ventana de contexto (como word2vec). La idea principal detrás de GloVe es que las co-ocurrencias de palabras en un corpus pueden proporcionar información semántica valiosa.

El modelo GloVe construye una matriz de co-ocurrencia global que tabula cuántas veces cada palabra aparece en el contexto de otras palabras dentro de un corpus. Luego, esta matriz se factoriza para reducir su dimensión, resultando en vectores de palabras más densos. El objetivo de la factorización es mantener la estructura semántica donde la distancia entre dos vectores de palabras refleje la similitud semántica entre las palabras correspondientes.

2 . PPMI (Positive Pointwise Mutual Information)
La PPMI es una técnica que se usa para calcular la asociación entre palabras basada en cuán frecuentemente aparecen juntas en comparación con cuán frecuentemente aparecen por separado. El "Pointwise Mutual Information" (PMI) de dos palabras mide la probabilidad de co-ocurrencia de las palabras en relación con las probabilidades de que cada palabra ocurra por sí sola. Sin embargo, PMI puede tener valores negativos, lo que puede ser problemático en algunos escenarios de modelado.

Para solucionarlo, se utiliza PPMI, donde todos los valores negativos de PMI se reemplazan por cero, enfocándose solo en las asociaciones positivas. En NLP, la PPMI a menudo se usa como una técnica de pre-procesamiento para construir matrices de características que luego pueden ser factorizadas (similar a SVD en GloVe) para obtener representaciones vectoriales de palabras.

Para implementar PPMI, primero construiremos una matriz de co-ocurrencia y luego convertiremos sus valores a PPMI. Usaremos numpy para las operaciones matemáticas y collections para construir la matriz de co-ocurrencia.




In [ ]:
import numpy as np
from collections import defaultdict, Counter
from itertools import product

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set(corpus)
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for i in range(len(corpus)):
        token = corpus[i]
        left = max(0, i-window_size)
        right = min(len(corpus), i+window_size+1)

        for j in range(left, right):
            if i != j:
                co_occurrences[token][corpus[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.maximum(
        np.log((co_matrix * total_sum) / (row_sums[:, None] * col_sums[None, :] + eps)),
        0
    )
    return ppmi

# Ejemplo de uso
corpus = "the quick brown fox jumps over the lazy dog".split()
co_matrix, vocab = co_occurrence_matrix(corpus, window_size=2)
ppmi = ppmi_matrix(co_matrix)

print(ppmi)


Implementar GloVe desde cero es más complejo debido a la optimización necesaria para ajustar los vectores de palabras. Sin embargo, puedes usar la biblioteca gensim, que tiene una implementación eficiente de GloVe. Utiliza el código realizado en clase.

In [ ]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

# Crear modelo Word2Vec con los mismos parámetros que GloVe
modelo = Word2Vec(sentences=[corpus], vector_size=100, window=5, min_count=1, sg=0, workers=4, epochs=10)

# Guardar y cargar el modelo (simulando una carga de GloVe)
model.wv.save_word2vec_format('model.bin')
glove_model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

# Usar el modelo
print(glove_model['fox'])  # Muestra el vector para la palabra "fox"


#### Ejercicios

1. Modifica el tamaño de la ventana de contexto en la función co_occurrence_matrix para diferentes valores (por ejemplo, 1, 3, y 5) y observa cómo cambia la matriz PPMI resultante. Analiza cómo el tamaño de la ventana afecta las relaciones semánticas capturadas (1 punto).
2. Implementa una función que identifique y muestre las palabras con mayor asociación (mayores valores PPMI) para una palabra dada. Utiliza esta función para explorar las relaciones semánticas de varias palabras clave en un corpus más grande (1 punto).
3. Usa la biblioteca gensim para entrenar un modelo GloVe con un corpus más grande (por ejemplo, un conjunto de datos de reseñas de productos o artículos de noticias). Ajusta diferentes hiperparámetros como el tamaño del vector, el tamaño de la ventana, y el número de iteraciones. Evalúa los vectores de palabras resultantes en tareas de analogía y similaridad (1 punto).
4. Realiza una comparación cualitativa y cuantitativa de las representaciones de palabras obtenidas a través de PPMI y GloVe. Considera aspectos como la capacidad de capturar sinónimos, antónimos y relaciones semánticas complejas. Discute en qué casos un método podría ser preferido sobre el otro (1 punto).

In [ ]:
## Tus respuestas

### Pregunta 3

El desarrollo de modelos de redes neuronales recurrentes (RNNs) ha sido fundamental en el avance del procesamiento de secuencias de tiempo y lenguaje natural. Estos modelos son especialmente útiles en tareas como el reconocimiento de voz, la traducción automática y la generación de texto. Sin embargo, las RNNs básicas enfrentan desafíos significativos, como la desaparición y la explosión del gradiente, que obstaculizan su capacidad para aprender dependencias a largo plazo en los datos. Las unidades de memoria de largo y corto plazo (LSTM) y las unidades recurrentes con compuertas (GRU) se desarrollaron como soluciones a estos problemas, mejorando la capacidad de las redes para aprender de datos secuenciales a largo plazo.

Una RNN básica procesa información secuencial mediante la actualización de su estado oculto con cada nuevo elemento de la secuencia. La naturaleza recurrente de estas redes les permite mantener una forma de 'memoria' sobre los elementos anteriores de la secuencia, utilizando la siguiente fórmula básica para actualizar el estado oculto en cada paso de tiempo $t$:

$$
h_t = \sigma(W_{ih} x_t + W_{hh} h_{t-1} + b_h)
$$

Donde $x_t$ es la entrada en el tiempo $t$, $h_t$ es el estado oculto en el tiempo $t$, $W_{ih}$ y $W_{hh}$ son los pesos de entrada y recurrentes, respectivamente, $b_h$ es el término de sesgo, y $\sigma$ es una función de activación no lineal como tanh o ReLU.


El entrenamiento de RNNs implica ajustar estos pesos mediante retropropagación a través del tiempo, lo que puede llevar a dos problemas principales:

1. **Desaparición del gradiente:** Si los gradientes de los pesos son muy pequeños, disminuyen exponencialmente a medida que se propagan hacia atrás a través de cada paso de tiempo. Esto hace que sea difícil para la RNN aprender dependencias a largo plazo, ya que los gradientes se vuelven insignificantes para ajustar los pesos efectivamente en pasos de tiempo anteriores.

2. **Explosión del gradiente:** En contraste, si los gradientes son demasiado grandes, pueden crecer exponencialmente durante la retropropagación, lo que lleva a actualizaciones de peso grandes e inestables, y por ende, a un modelo que diverge y no aprende de manera efectiva.

#### Unidad de memoria de largo y corto plazo (LSTM)

Para abordar estos problemas, se introdujeron las LSTMs, que incorporan un diseño más complejo que permite controlar el flujo de información. Las LSTMs utilizan varias "puertas" para regular tanto el almacenamiento como la eliminación de información en el estado de la celda:

- **Puerta de olvido $(f_t)$** decide qué parte de la información anterior se mantiene:
  $$
  f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
  $$

- **Puerta de entrada ($i_t$) y candidato de celda ($\tilde{c}_t$)** deciden qué nueva información se añade al estado de la celda:

  $$
  i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
  $$
  $$
  \tilde{c}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)
  $$

- **Actualización del estado de la celda ($c_t$)** combina la información antigua y nueva:
  $$
  c_t = f_t \ast c_{t-1} + i_t \ast \tilde{c}_t
  $$

- **Puerta de salida ($o_t$)** y el estado oculto resultante ($h_t$) que determina qué parte del estado de la celda afectará la salida:
  $$
  o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
  $$
  $$
  h_t = o_t \ast \tanh(c_t)
  $$

#### Unidad recurrente compuerta (GRU)

Las GRUs simplifican la arquitectura de las LSTMs combinando las puertas de entrada y olvido en una sola puerta de actualización y omitiendo el uso de un estado de celda separado:

- **Puerta de actualización ($z_t$)** decide cuánto del estado anterior se debe mantener:
  $$
  z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z)
  $$

- **Puerta de reinicio ($r_t$)** decide cuánto del pasado se debe olvidar antes de calcular el nuevo candidato de estado:
  $$
  r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r)
  $$

- **Candidato de estado oculto ($\tilde{h}_t$)** y la actualización del estado oculto:
  $$
  \tilde{h}_t = \tanh(W_h \cdot [r_t \ast h_{t-1}, x_t] + b_h)
  $$
  
  $$
  h_t = (1 - z_t) \ast h_{t-1} + z_t \ast \tilde{h}_t
  $$


#### Ejercicios

1. ¿Qué papel juegan los reguladores como dropout o L2 regularization específicamente en el contexto de RNNs y LSTM para evitar el sobreajuste en tareas de modelado de lenguaje? (1 punto)
2. Considerando la complejidad computacional de BPTT, ¿cuáles son las limitaciones prácticas cuando se usa con RNNs en secuencias muy largas? ¿Cómo podrías mitigar estos problemas en un entorno de producción? (1 punto)


In [ ]:
## Parte 3

import torch
from torch import nn
import torch.nn.functional as F

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_hidden = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.tanh(self.input_to_hidden(combined))
        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size

        # Gates
        self.input_to_inputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_forgetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_outputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_cellgate = nn.Linear(input_state + hidden_size, hidden_size)

    def forward(self, input, hidden, cell):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        input_gate = torch.sigmoid(self.input_to_inputgate(combined))
        forget_gate = torch.sigmoid(self.input_to_forgetgate(combined))
        output_gate = torch.sigmoid(self.input_to_outputgate(combined))
        cell_gate = torch.tanh(self.input_to_cellgate(combined))

        # Update cell state
        cell = forget_gate * cell + input_gate * cell_gate
        hidden = output_gate * torch.tanh(cell)

        return hidden, cell

    def initHidden(self):
        return torch.zeros(1, self.hidden_size), torch.zeros(1, self.hidden_size)


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_updategate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_resetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_newgate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        update_gate = torch.sigmoid(self.input_to_updategate(combined))
        reset_gate = torch.sigmoid(self.input_to_resetgate(combined))
        new_gate = torch.tanh(self.input_to_newgate(combined * reset_gate))

        # Update hidden state
        hidden = update_gate * hidden + (1 - update_gate) * new_gate

        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden.store)


Extiende la implementación de LSTM para incluir embeddings de palabras y una capa de clasificación, y entrenar el modelo en una tarea de predicción de la siguiente palabra en secuencias de texto (3 puntos).

- Agrega una capa de embedding al modelo LSTM para procesar entradas de texto.
- Incluye una capa de salida que mapee el estado oculto a las predicciones de palabras.
- Implementa una función de pérdida adecuada para la clasificación de palabras.
- Preprocesa  un corpus de texto grande (utiliza los datos dados en clase por ejemplo) para convertir texto a índices utilizando un vocabulario predefinido.
- Genera datos de entrenamiento como pares de secuencias de entrada y palabras objetivo.

Realiza un análisis de sensibilidad de los hiperparámetros en modelos LSTM y GRU para entender su impacto en la capacidad de aprendizaje de dependencias a largo plazo en textos (3 puntos)

- Selecciona un corpus de texto y prepara datos para el entrenamiento de modelos de lenguaje basados en LSTM y GRU.
- Experimenta con diferentes valores para los hiperparámetros como el tamaño de las puertas, la tasa de aprendizaje, el tamaño del estado oculto y la longitud de BPTT.
- Utiliza técnicas como validación cruzada para evaluar el impacto de estos cambios en la precisión del modelo y en su capacidad para generar texto coherente.
- Analiza cómo la modificación de los parámetros de las puertas y la longitud de BPTT afecta la estabilidad del entrenamiento y la convergencia del modelo.

In [ ]:
## Tus respuestas

### Pregunta 4
El script proporcionado es un ejemplo completo de cómo implementar un modelo de red neuronal recurrente (RNN) utilizando PyTorch para generar texto de manera automática.

In [ ]:
# Importación de librerías necesarias para trabajar con tensores y redes neuronales.
import torch
from torch import nn
import numpy as np

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')


La longitud mayor tiene 15 caracteres
Secuencia entrada: hey how are yo
Secuencia objetivo: ey how are you
Secuencia entrada: good i am fine
Secuencia objetivo: ood i am fine 
Secuencia entrada: have a nice da
Secuencia objetivo: ave a nice day
Forma de entrada: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)
GPU no disponible, CPU es usada
Epoch: 10/100............. Loss: 2.4314
Epoch: 20/100............. Loss: 2.0728
Epoch: 30/100............. Loss: 1.6524
Epoch: 40/100............. Loss: 1.2482
Epoch: 50/100............. Loss: 0.9012
Epoch: 60/100............. Loss: 0.6362
Epoch: 70/100............. Loss: 0.4544
Epoch: 80/100............. Loss: 0.3299
Epoch: 90/100............. Loss: 0.2443
Epoch: 100/100............. Loss: 0.1868


'good i am fine '

#### Ejercicios:

1. Modifica el modelo existente para que funcione como un autoencoder. Esto implica que el modelo debe aprender a codificar una secuencia de entrada en un vector de características (estado oculto) y luego decodificar ese vector de vuelta a la secuencia original (1.5 puntos).
     - Implementa las capas de codificación y decodificación dentro del mismo modelo.
     - Experimenta  con diferentes estructuras como LSTM para mejorar la retención de información.
     - Mide la calidad de la reconstrucción del texto y la eficiencia de compresión.

2. Utiliza el modelo RNN actual y modifícalo para introducir secuencias más largas. Monitoriza los gradientes durante el entrenamiento para detectar signos de desaparición o explosión. (1.5 puntos)
    - Implementa el  clipping de gradiente para prevenir la explosión del gradiente.
    - Reemplaza la RNN por LSTM para abordar la desaparición del gradiente.
    - Utiliza técnicas de visualización para observar la magnitud de los gradientes a lo largo de varias épocas.
3. Implementa el dropout en las capas recurrentes y comparar los resultados. (1 punto)

    - Ajusta el parámetro de weight decay en el optimizador y observar el efecto sobre el overfitting.
    - Aplica early stopping basado en la validación del loss para detener el entrenamiento antes de que el modelo comience a sobreajustarse.


In [ ]:
## Tus respuestas

# 1. Modifica el modelo existente para que funcione como un autoencoder.
# Esto implica que el modelo debe aprender a codificar una secuencia de entrada en un
# vector de características (estado oculto) y luego decodificar ese vector de vuelta
# a la secuencia original (1.5 puntos).

# Implementa las capas de codificación y decodificación dentro del mismo modelo.
# Experimenta con diferentes estructuras como LSTM para mejorar la retención de información.
# Mide la calidad de la reconstrucción del texto y la eficiencia de compresión.


# Importación de librerías necesarias para trabajar con tensores y redes neuronales.
import torch
from torch import nn
import numpy as np

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')


La longitud mayor tiene 15 caracteres
Secuencia entrada: hey how are yo
Secuencia objetivo: ey how are you
Secuencia entrada: good i am fine
Secuencia objetivo: ood i am fine 
Secuencia entrada: have a nice da
Secuencia objetivo: ave a nice day
Forma de entrada: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)
GPU no disponible, CPU es usada
Epoch: 10/100............. Loss: 2.5097
Epoch: 20/100............. Loss: 2.0451
Epoch: 30/100............. Loss: 1.6461
Epoch: 40/100............. Loss: 1.2529
Epoch: 50/100............. Loss: 0.9024
Epoch: 60/100............. Loss: 0.6297
Epoch: 70/100............. Loss: 0.4415
Epoch: 80/100............. Loss: 0.3150
Epoch: 90/100............. Loss: 0.2301
Epoch: 100/100............. Loss: 0.1735


'good i am fine '

In [ ]:
# 2. Utiliza el modelo RNN actual y modifícalo
# para introducir secuencias más largas.
# Monitoriza los gradientes durante el entrenamiento
#  para detectar signos de desaparición o explosión. (1.5 puntos)

# a. Implementa el clipping de gradiente para prevenir la explosión del gradiente.

class RNNOptimizer(object):
    # Constructor de la clase optimizador para redes neuronales recurrentes.
    # Args:
    #   gradient_clipping (bool): Indica si se aplica clipping a los gradientes,
    #                             activado por defecto.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        self.lr = lr  # Almacena la tasa de aprendizaje.
        self.gradient_clipping = gradient_clipping  # Almacena la configuración de clipping.
        self.first = True  # Variable auxiliar, posiblemente para controlar la primera actualización.

    # Método que ejecuta un paso de optimización sobre todos los parámetros del modelo.
    def step(self) -> None:
        # Itera sobre cada capa del modelo.
        for layer in self.model.layers:
            # Itera sobre cada parámetro de la capa.
            for key in layer.params.keys():
                # Si el clipping de gradientes está activado, aplica clipping.
                if self.gradient_clipping:
                    # Los gradientes se limitan a estar entre -2 y 2 para evitar la explosión de gradientes.
                    np.clip(layer.params[key]['deriv'], -2, 2, layer.params[key]['deriv'])

                # Llama a la regla de actualización para ajustar el valor del parámetro
                # usando la tasa de aprendizaje y el gradiente actual.
                self._update_rule(param=layer.params[key]['value'],
                                  grad=layer.params[key]['deriv'])

    # Método abstracto para definir la regla de actualización de parámetros.
    # Debe ser implementado por subclases.
    def _update_rule(self, **kwargs) -> None:
        raise NotImplementedError("Este método debe ser implementado por subclases.")

class SubClassOptimizer(RNNOptimizer):
    # Constructor de la subclase de optimizador para redes neuronales recurrentes.
    # Args:
    #   gradient_clipping (bool): Indica si se aplica clipping a los gradientes,
    #                             activado por defecto.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        super().__init__(lr, gradient_clipping)  # Llama al constructor de la clase base.

    # Método que define la regla de actualización de parámetros para redes neuronales recurrentes.
    def _update_rule(self, param: np.ndarray, grad: np.ndarray) -> None:
        # Implementación de la regla de actualización de parámetros para redes neuronales recurrentes.
        # ...
        pass

# Instancia del optimizador con clipping de gradientes activado.
optimizer = SubClassOptimizer(gradient_clipping=True)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout=0.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1, dropout=0.5)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')




In [ ]:
# 3. Implementa el dropout en las capas recurrentes y comparar los resultados. (1 punto)

# Ajusta el parámetro de weight decay en el optimizador y observar el efecto sobre el
# overfitting.
# Aplica early stopping basado en la validación del loss para detener el entrenamiento antes
# de que el modelo comience a sobreajustarse.

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout=0.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1, dropout=0.5)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')

# Implementación de dropout en las capas recurrentes.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout=0.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con dropout en las capas recurrentes.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1, dropout=0.5)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')


La longitud mayor tiene 15 caracteres
Secuencia entrada: hey how are yo
Secuencia objetivo: ey how are you
Secuencia entrada: good i am fine
Secuencia objetivo: ood i am fine 
Secuencia entrada: have a nice da
Secuencia objetivo: ave a nice day
Forma de entrada: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)
GPU no disponible, CPU es usada
Epoch: 10/100............. Loss: 2.4890
Epoch: 20/100............. Loss: 2.0581
Epoch: 30/100............. Loss: 1.6449
Epoch: 40/100............. Loss: 1.2527
Epoch: 50/100............. Loss: 0.9131


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 60/100............. Loss: 0.6512
Epoch: 70/100............. Loss: 0.4671
Epoch: 80/100............. Loss: 0.3410
Epoch: 90/100............. Loss: 0.2557
Epoch: 100/100............. Loss: 0.1980
Epoch: 10/100............. Loss: 2.4387
Epoch: 20/100............. Loss: 2.1774
Epoch: 30/100............. Loss: 1.7693
Epoch: 40/100............. Loss: 1.3152
Epoch: 50/100............. Loss: 0.9320
Epoch: 60/100............. Loss: 0.6467
Epoch: 70/100............. Loss: 0.4483
Epoch: 80/100............. Loss: 0.3170
Epoch: 90/100............. Loss: 0.2301
Epoch: 100/100............. Loss: 0.1736


'good i am fine '